# Data Acquisition Notebook
- Notebook for code to be refactored for ```build_database.py```
- Database will hold 2 most recent congresses

## ProPublica Configuration

In [ ]:
# API Key
import configparser

In [ ]:
config = configparser.ConfigParser()
config.read('../auth/config.ini')
PROPUBLICA_API_KEY = config.get('propublica', 'PROPUBLICA_API_KEY')

In [ ]:
API_ROOT = 'https://api.propublica.org/congress/v1/'
header = {'X-API-Key': f'{PROPUBLICA_API_KEY}'}

## Get Initial Congress and Clean Data/Select Features

In [ ]:
# Read/Scrape data
import requests
import json
import re
from bs4 import BeautifulSoup
from datetime import datetime

from pprint import pprint

In [ ]:
def get_house_ids(congress, header):
    '''
    Function to get house members' ProPublica ID by congress number
    '''

    call_string = API_ROOT + f'{congress}/house/members.json'
    r = requests.get(call_string, headers=header)
    result = r.json()['results'][0]['members']
    member_ids = [ member['id'] for member in result ]
    
    return member_ids

In [ ]:
# House member ids of the 116-117th Congress (2018-2022)
ids_116 = get_house_ids(116)
ids_117 = get_house_ids(117)

In [ ]:
# Create list of unique IDs to remove redundant API calls
all_ids = list(set(ids_116 + ids_117))

In [ ]:
def get_member_data(member, header):
    '''
    Function to get house member's data
    '''
    
    call_string = API_ROOT + f'members/{member}.json'
    r = requests.get(call_string, headers=header)
    result = r.json()['results'][0]
    
    return result

In [ ]:
# Sample
sample_id = all_ids[0]
sample = get_member_data(sample_id)
for k, v in list(sample.items())[:5]:
    print(f'{k}: {v}')

In [ ]:
# Create generator, split list in half to ensure under batch write limit
half = len(all_ids)/2
members1 = ( get_member_data(member) for member in all_ids[:half] )
members2 = ( get_member_data(member) for member in all_ids[half:] )

In [ ]:
def to_date(s):
    date = datetime.strptime(s, '%Y-%m-%d')
    return date

In [ ]:
def member_cleaner(member):
    '''
    Function to keep relevent information on congress member
    '''
    
    roles = member['roles']
    roles.sort(key=lambda x: x['congress'], reverse=True)
    fec_id = roles[0]['fec_candidate_id'] # Most recent FEC candidate ID
    
    mem_dict = {
        '_id': member['id'],
        'bio': {
            'first_name': member['first_name'],
            'middle_name': member['middle_name'],
            'last_name': member['last_name'],
            'dob': to_date(member['date_of_birth']),
            'gender': member['gender'],
            'current_party': member['current_party'],
        },
        'activity': {
            'last_updated': to_date(member['last_updated'][:10]), # Ignore time
            'in_office': member['in_office'],
        },
        'other_ids': {
            'google_id': member['google_entity_id'],
            'votesmart_id': member['votesmart_id'],
            'govtrack_id': member['govtrack_id'],
            'cspan_id': member['cspan_id'],
            'crp_id': member['crp_id'],
            'fec_id': fec_id,
        },
        'roles': member['roles'],   
    }
    
    return mem_dict

## Inserting into Google Firestore

In [ ]:
# Database connection and authentication
import firebase_admin
from firebase_admin import credentials, firestore
# from google.cloud.firestore_v1.batch import WriteBatch

In [ ]:
GCP_AUTH_PATH = config.get('firebase', 'GCP_AUTH_PATH')
cred = credentials.Certificate(GCP_AUTH_PATH)
app = firebase_admin.initialize_app(cred)

In [ ]:
# Database
db = firestore.client()

In [ ]:
members_insert1 = map(member_cleaner, members1)
members_insert2 = map(member_cleaner, members2)

In [ ]:
def batch_insert_members(members, db):
    '''
    Function to batch insert house members into database
    '''
    
    batch = db.batch()
    members_len = 0
    for member in members:
        _id = member['_id']
        insert_ref = db.collection("reps").document(f"{_id}")
        batch.set(insert_ref, member)
        members_len += 1
            
    batch.commit()
    
    batch_len = len(batch.write_results)
    f_string = f'Batch Length: {batch_len}, Members Length: {members_len}'
    assert batch_len == members_len, f_string
    
    print f'{batch_len} members inserted'

In [ ]:
batch_insert_members(members_insert1, db)
batch_insert_members(members_insert2, db)